In [4]:
# work with saved historic data to minimize the number of api requests

import pandas as pd
import os

symbol = 'btc'

tick_data = pd.read_csv(symbol+'_USD_2021-05-05.csv', index_col=1, infer_datetime_format=True, parse_dates=True)

tick_data.sort_index(inplace=True)

# implement re-sampling
'''
bar lengths are in seconds
5 min: 300s, 10 min: 600s, 20 min: 1200s, 40 min: 2400s, 60min: 3600s
3 hr: 10800s, 6hr: 21600s, 9hr: 32400s, 12hr: 43200s, 15hr: 54000, 18hr: 64800
24 hr: 86400s, 48 hr: 172800s, 72 hr: 259200s, 120 hr: 432000s
'''
df = pd.DataFrame()
bar = '21600s'
df = tick_data['<CLOSE>'].resample(bar, label='right').last().ffill()
df.head()
pd.plotting.autocorrelation_plot(df[-60:])

<AxesSubplot:xlabel='Lag', ylabel='Autocorrelation'>

In [2]:
df.head()

Date
2021-01-04    33002.53643
2021-01-05    31431.61228
2021-01-06    34433.60651
2021-01-07    36275.75635
2021-01-08    39713.50786
Freq: 86400S, Name: <CLOSE>, dtype: float64

In [5]:
import time
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

sub = pd.DataFrame(df)

# analyze data sub-set

i = 3 # Number of ticks to use for simple moving average (SMA)
t = str(time.time()) #datetime stamp

sub['SMA'] = sub['<CLOSE>'].rolling(i).mean().shift(i)
sub.plot(figsize=(10,6), lw=0.75, title='BTC-USD', ylabel='USD/BTC')
#plt.savefig('1-BTC-USD_SMA_{}.png'.format(t))
#plt.show()
sub.head()

,<CLOSE>,SMA
Date,,
2021-01-03 06:00:00,33002.53643,NaN
2021-01-03 12:00:00,33002.53643,NaN
2021-01-03 18:00:00,33002.53643,NaN
2021-01-04 00:00:00,33002.53643,NaN
2021-01-04 06:00:00,31431.61228,NaN


In [6]:
import pandas as pd
import numpy as np

# vectorized backtesting

ptc = 0.005 # proportional transactional costs - Coinbase Pro charges 0.5% per transaction

data = pd.DataFrame()
data['close'] = sub['<CLOSE>']
data['returns'] = np.log(data['close']/data['close'].shift(1))

data.dropna(inplace=True)

data = data[-60:]

lags = 13

cols = []

for lag in range(1, lags+1):
    col = 'lag_{}'.format(lag)
    data[col] = data['returns'].shift(lag)
    cols.append(col)

data.dropna(inplace=True)

data[cols] = np.where(data[cols] >= 0, 1, 0)

data['direction'] = np.where(data['returns'] > 0, 1, -1)

data[cols + ['direction']].head()

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,direction
Date,,,,,,,,,,,,,,
2021-04-23 18:00:00,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,-1
2021-04-24 00:00:00,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,-1
2021-04-24 06:00:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1
2021-04-24 12:00:00,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,-1
2021-04-24 18:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,-1


In [39]:
data.to_csv('backtest_data.csv')

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from datetime import datetime
import pickle

model = SVC(C=1, kernel='linear', gamma='auto')

split = int(len(data) * 0.50)

train = data.iloc[:split].copy()

model.fit(train[cols], train['direction'])

# persisting the model object
t = str(time.time()) #datetime stamp
pickle.dump(model, open('algorithm-{}.pkl'.format(t), 'wb'))

In [16]:
accuracy_score(train['direction'], model.predict(train[cols]))

0.9130434782608695

In [17]:
test = data.iloc[split:].copy()

test['position'] = model.predict(test[cols])
#test['position'] = test['direction']         # naive prediction

test[['position','direction']].tail()

,position,direction
Date,,
2021-05-04 06:00:00,1,-1
2021-05-04 12:00:00,-1,-1
2021-05-04 18:00:00,-1,-1
2021-05-05 00:00:00,-1,-1
2021-05-05 06:00:00,1,1


In [18]:
accuracy_score(test['direction'], test['position'])

0.9583333333333334

In [19]:
test['strategy'] = test['position'] * test['returns']
sum(test['position'].diff() != 0)

10

In [20]:
test['strategy_tc'] = np.where(test['position'].diff() != 0, test['strategy'] - ptc, test['strategy'])

In [21]:
test[['returns', 'strategy', 'strategy_tc']].sum().apply(np.exp)

returns        1.074960
strategy       1.123150
strategy_tc    1.068374
dtype: float64

In [22]:
t = str(time.time())
test[['returns', 'strategy', 'strategy_tc']].cumsum().apply(np.exp).plot(figsize=(10,6))
plt.savefig('1-BTC-USD_StratCompare_{}.png'.format(t))
plt.show()

In [94]:
import pandas as pd
from datetime import datetime
t = str(time.time()) #datetime stamp #datetime stamp
test.to_csv('backtest_{}.csv'.format(t))

In [5]:
# load the ARIMA model from file
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMAResults
import numpy as np

model = ARIMAResults.load('arima_model.pkl')
print(loaded.params)
data = np.load('arima_data.npy')
last_obs = np.load('arima_obs.npy')
print(last_obs)

[-1.04422399e+00 -4.42645035e-02 -7.95014164e-04 -9.99203632e-01
  4.22141698e+06]
[57252.70218]


In [11]:
predictions = model.predict(start=len(data), end = len(data))
yhat = predictions[0] + last_obs[0]
print('Prediction: %f' % yhat)

Prediction: 57420.468098
